# Belarus COVID-19 status

The first case in Belarus is dated Feb 27 ([According to govt report](http://minzdrav.gov.by/ru/sobytiya/v-belarusi-zaregistrirovan-zavoznoy-sluchay-koronavirusa/))

In [1]:
import pandas as pd
import numpy as np
import altair as alt

default_chart_properties = {
    'width': 550,
    'height': 400
}

SOURCE_NAME = 'COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University'
TIME_SERIES_URL = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_{}_global.csv'

In [2]:
def load_series(country, key):
    df = pd.read_csv(TIME_SERIES_URL.format(key))

    df = df.loc[df['Country/Region'] == country] \
        .drop(columns=['Lat', 'Long', 'Province/State', 'Country/Region']) \
        .melt() \
        .rename(columns={'variable': 'date', 'value': 'total_cases'})

    df['date'] = pd.to_datetime(df.date)
    df = df.sort_values(by='date')

    return df

In [3]:
confirmed_cases = load_series('Belarus', 'confirmed')
deaths_cases = load_series('Belarus', 'deaths')
recovered_cases = load_series('Belarus', 'recovered')

print('Case series as for {}'.format(
    confirmed_cases.iloc[-1]['date'].strftime('%d.%m.%Y')))

Case series as for 23.05.2020


### Total Cases

In [4]:
total_series = pd.concat([confirmed_cases, recovered_cases, deaths_cases],
                         keys=['Confirmed', 'Recovered', 'Deaths'])

total_series = total_series[total_series['total_cases'] > 0]

alt.Chart(total_series.reset_index()).mark_area().encode(
    x=alt.X('date:T', title='Date'),
    y=alt.Y('total_cases:Q', title='Total cases', stack=None),
    color=alt.Color('level_0:N', title='Source'),
).properties(**default_chart_properties)

alt.Chart(...)

### New cases

In [5]:
new_cases_growth = confirmed_cases.copy()
new_cases_growth['total_cases'] = new_cases_growth['total_cases'].diff()
new_cases_growth = new_cases_growth[new_cases_growth['total_cases'] > 0]

alt.Chart(new_cases_growth).mark_bar().encode(
    x=alt.X('date:T', title='Date'),
    y=alt.Y('total_cases:Q', title='New cases'),
).properties(**default_chart_properties)

alt.Chart(...)

### Death rate

In [6]:
death_rate = confirmed_cases[['date']].copy()
death_rate['death_rate'] = deaths_cases['total_cases'] / confirmed_cases['total_cases'] * 100
death_rate = death_rate[death_rate['death_rate'] == death_rate['death_rate']]

max_index = death_rate['death_rate'].argmax()
max_entry = death_rate.iloc[max_index]
print('Max death rate: {0:.2f}% on {1}'.format(max_entry['death_rate'], max_entry['date'].strftime('%d.%m.%Y')))

last_entry = death_rate.iloc[-1]
print('Latest death rate: {0:.2f}% on {1}'.format(last_entry['death_rate'], last_entry['date'].strftime('%d.%m.%Y')))

alt.Chart(death_rate).mark_bar().encode(
    x=alt.X('date:T', title = 'Date'),
    y=alt.Y('death_rate:Q', title = 'Death rate, %'),
).properties(**default_chart_properties)

Max death rate: 1.86% on 06.04.2020
Latest death rate: 0.55% on 23.05.2020


alt.Chart(...)

### Ministry of Public Health forecast
This data is approximation to the forecast was spotted by journalists during president's speech ([Video at 1:24](https://mirtvpc.cdnvideo.ru/mirtv/1588505558104.mp4?fbclid=IwAR39k3nFWNhs8XxPMp8yy_XDfpIyw-B4iccRIK5lU6h5u2eqjUqc34dSPSc))

In [7]:
forecast_data_raw = [
    ['5/3/20', 16705],
    ['5/4/20', 17791],
    ['5/5/20', 18798],
    ['5/6/20', 19770],
    ['5/7/20', 20602],
    ['5/8/20', 21567],
    ['5/9/20', 22606],
    ['5/10/20', 23539],
    ['5/11/20', 24339],
    ['5/12/20', 25297],
    ['5/13/20', 26168],
    ['5/14/20', 27084],
    ['5/15/20', 28019],
    ['5/16/20', 28890],
    ['5/17/20', 29836],
    ['5/18/20', 30658],
    ['5/19/20', 31561],
    ['5/20/20', 32369],
    ['5/21/20', 33237],
    ['5/22/20', 34118],
    ['5/23/20', 35026],
    ['5/24/20', 35894],
    ['5/25/20', 36611],
    ['5/26/20', 37431],
    ['5/27/20', 38166],
    ['5/28/20', 38771],
    ['5/29/20', 39246],
    ['5/30/20', 39846],
    ['5/31/20', 40271],
]

forecast_data = pd.DataFrame(forecast_data_raw, columns=['date', 'total_cases'])
forecast_data['date'] = pd.to_datetime(forecast_data.date)

In [8]:
confirmed_cases_started_from_may = confirmed_cases[
    (confirmed_cases['total_cases'] > 0) & \
    (confirmed_cases['date'] >='4/28/2020') & \
    (confirmed_cases['date'] <='6/5/2020')
]

confirmed_cases_chart = alt.Chart(
    confirmed_cases_started_from_may,
    title='Real cases vs. forecast for May 3-31'
).mark_bar().encode(
    x=alt.X('date:T', title='Date'),
    y=alt.Y('total_cases:Q', title='Total cases'),
)

forecast_cases_chart = alt.Chart(
    forecast_data,
).mark_line(
    color='red'
).encode(
    x=alt.X('date:T'),
    y=alt.Y('total_cases:Q'),
)

(confirmed_cases_chart + forecast_cases_chart).properties(
    **default_chart_properties)

alt.LayerChart(...)

### Recovering period
Calculating an average period need for recovering from COVID-19. From the statistic data, it's a period on which recovered + deaths get equal to previously confirmed cases.

In [9]:
rd_cases = recovered_cases.copy()
rd_cases['total_cases'] += deaths_cases['total_cases']

recovery_days = []
for rd_case in rd_cases.values[::-1]:
    first_confirmed_idx = np.argmax(
        confirmed_cases['total_cases'] >= rd_case[1])
    
    if first_confirmed_idx == 0:
        break
    
    recovery_days.insert(0, (rd_case[0] - confirmed_cases.loc[first_confirmed_idx]['date']).days)

median_recovery_days = int(np.median(recovery_days[-14:]))

print('COVID-19 recovering period last 2 weeks (median) is {} days'.format(median_recovery_days))

COVID-19 recovering period last 2 weeks (median) is 20 days
